In [ ]:
! pip install pandas
! pip install boto3
! pip install watchtower
! pip install s3fs==0.4.2
! pip install pyathena
! pip install matplotlib
! pip install scipy
! pip install ipywidgets
! pip install scikit-learn

! conda install -c conda-forge --yes implicit 

! pip install --upgrade jupyter_client # useful to make ipywidgets work properly when fitting data with implicit

# pip install git+https://gitlab.com/cloena/cloena-aws-tools.git

In [1]:
import pandas as pd

import implicit

from aws_tools import athena_tools, s3_tools

import matplotlib

import re

import scipy

from ipywidgets import FloatProgress

import numpy as np

from sklearn.preprocessing import MinMaxScaler

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/aws_tools/project_config.py:18: UserWarning: no aws_config.json file found
  warnings.warn('no aws_config.json file found')


In [17]:
_v = '_v0.2'

In [3]:
query = """
SELECT
    LINE_ACCOUNT_ID AS ID_USER -- account ID
    ,EVENT_SLOT_TYPE AS TYPE_SLOT -- Feature 62M, Trailer2M, Promo 114, Promotion 6M
    ,CI_ASSET_TITLE AS TITLE -- human readable title name
    ,CI_ASSET_TYPE AS TYPE_ASSET -- e.g. Film, Music, etc.
    ,VIEW_TIME_ST AS VIEW_DATE -- time of purchase
    ,EDITORIAL_VERSION_ID AS ID_EDITORIAL -- identifies multiple instances of same film/season/episode, e.g. rent/purchase & SD/HD
FROM
    bt_home_datamart.l_edw_vod_views;
"""

# WHERE (ENTITLEMENT_TYPE LIKE 'EVOD' OR ENTITLEMENT_TYPE LIKE 'TVOD') -- can be used to exclude PPV, but better to include it and then remove anything no longer available from the catalogue

data = athena_tools.AthenaQuerier().execute_query(query=query)

2020-06-24 14:10:19,651 [INFO ]  starting Athena query ...
2020-06-24 14:15:48,493 [INFO ]  loading 0224f14d-1f01-4765-bd6f-ef243ea7111c.csv
2020-06-24 14:20:04,599 [INFO ]  Athena query complete: returning dataframe


In [4]:
# keep data is only Feature to exclude what is traler/promot/promotion
data = data[data['TYPE_SLOT'] == 'Feature']

In [6]:
# Drop any nan users or editorial ids
data = data[(~data['ID_USER'].isnull()) & (~data['ID_EDITORIAL'].isnull())]

In [7]:
# need a temporary table to bring from the product catalogue the ID_PRODUCT to the views list by matching on ID_EDITORIAL
query = """
SELECT
    PRODUCT_GUID AS ID_PRODUCT -- item ID
    ,EDITORIAL_VERSION_ID AS ID_EDITORIAL -- identifies multiple instances of same film/season/episode, e.g. rent/purchase & SD/HD
    ,CI_PARENTGUID AS ID_PARENT -- if present, can connect episode to season by matching with season PRODUCT_ID
FROM
    bt_home_datamart.l_edw_vod_products;
"""

tmp = athena_tools.AthenaQuerier().execute_query(query=query)

data = data.merge(tmp.drop_duplicates('ID_EDITORIAL'), left_on='ID_EDITORIAL', right_on='ID_EDITORIAL', suffixes=('', ''))

del tmp

2020-06-24 14:24:41,866 [INFO ]  starting Athena query ...
2020-06-24 14:24:51,244 [INFO ]  loading 2d9f40ca-63cc-4d0b-b655-797edebd5eca.csv
2020-06-24 14:24:52,061 [INFO ]  Athena query complete: returning dataframe


In [8]:
query = """
SELECT
    PRODUCT_GUID AS ID_PRODUCT -- item ID
    ,CI_TITLE AS TITLE --  human readable title
    ,CI_TYPE AS TYPE -- type, like film/music/episode/season/collection
    ,CI_AVAILABLE_END_DT AS END_DATE -- date until availability of item
    ,EDITORIAL_VERSION_ID AS ID_EDITORIAL -- identifies multiple instances of same film/season/episode, e.g. rent/purchase & SD/HD
    ,CI_PARENTGUID AS ID_PARENT -- if present, can connect episode to season by matching with season PRODUCT_ID
    ,GENRE 
    ,RATING
FROM
    bt_home_datamart.l_edw_vod_products;
"""

cat = athena_tools.AthenaQuerier().execute_query(query=query)

2020-06-24 14:26:38,128 [INFO ]  starting Athena query ...
2020-06-24 14:26:49,568 [INFO ]  loading eee67f9c-be72-4f80-9dab-293490c85baa.csv
2020-06-24 14:26:51,803 [INFO ]  Athena query complete: returning dataframe


In [9]:
print(data.head())
print(cat.head())

        ID_USER TYPE_SLOT                                              TITLE  \
0  BBEU35442404   Feature  Deadliest Catch (Specials) S15-E20 Time And Ti...   
1  BBEU32222677   Feature  Deadliest Catch (Specials) S15-E20 Time And Ti...   
2  BBEU23135596   Feature  Deadliest Catch (Specials) S15-E20 Time And Ti...   
3  BBEU25732041   Feature  Deadliest Catch (Specials) S15-E20 Time And Ti...   
4  BBEU31947617   Feature  Deadliest Catch (Specials) S15-E20 Time And Ti...   

  TYPE_ASSET           VIEW_DATE ID_EDITORIAL ID_PRODUCT           ID_PARENT  
0    episode 2019-09-27 19:44:41  BBJ2311169A    8565378  movida_10051693_HD  
1    episode 2019-09-29 14:20:35  BBJ2311169A    8565378  movida_10051693_HD  
2    episode 2019-09-29 23:11:18  BBJ2311169A    8565378  movida_10051693_HD  
3    episode 2019-09-29 15:34:33  BBJ2311169A    8565378  movida_10051693_HD  
4    episode 2019-09-29 07:02:53  BBJ2311169A    8565378  movida_10051693_HD  
            ID_PRODUCT                       

In [10]:
print(data.shape)
print(cat.shape)

(56706624, 8)
(447552, 8)


In [11]:
# Left join the parent id in data with the product id from the catalogue dataframe
# By joining something that has a parent with the parent effectively connects episodes to the seasons (once we have bran info, we will perform that extra join step)
# we then fill the empty cells of editorial versions (those that are not seasons) with the same editorial data so that we have a single column with editorial id for each purchase/rental

# first we fill any empty PARENT_GUID with PRODUCT_ID to make the merge fully work below
data['ID_PARENT'].fillna(data['ID_PRODUCT'], inplace=True)

data = data.merge(cat, left_on='ID_PARENT', right_on='ID_PRODUCT', suffixes=('_data', ''))

del cat

# From here for editorial we will look at the cat version as it will have the editorial ID of the season
# Drop duplicate or obsolete columns
data.drop(columns=['ID_PRODUCT_data', 'TITLE_data', 'ID_EDITORIAL_data', 'ID_PARENT_data', 'ID_PRODUCT', 'ID_PARENT'], inplace=True)

# we don't drop items that have an END_DATE in the past because we want to create the model, but will need to set those as zero to make sure we do not recommend anything that is not available anymore

In [14]:
print(data.shape) 
data.sort_values(by='VIEW_DATE', ascending=False).head()

(56716699, 10)


,ID_USER,TYPE_SLOT,TYPE_ASSET,VIEW_DATE,TITLE,TYPE,END_DATE,ID_EDITORIAL,GENRE,RATING
56023141,BBEU37258276,Feature,episode,2020-06-22 23:59:46,Families Of The Mafia Series 1,season,2020-07-18 22:59:00,movida_10081516,Reality,15
55307106,BBEU28779867,Feature,episode,2020-06-22 23:59:44,Dispatches From Elsewhere Series 1,season,2020-07-23 22:59:00,movida_10079820,Drama,18
11725323,BBEU21999981,Feature,episode,2020-06-22 23:59:43,Peppa Pig Series 3,season,2020-10-31 23:59:00,movida_26616,2 - 5 Years,u
56606796,BBEU03372506,Feature,episode,2020-06-22 23:59:42,Blaydes v Volkov,collection,2020-06-26 22:30:00,movida_10084415,UFC,18
55260706,BBEU19739058,Feature,episode,2020-06-22 23:59:42,Secrets In The Ice Series 1,season,2020-07-15 21:30:00,movida_10082966,Documentary,pg


In [15]:
data['RATING'].value_counts()

u     30718766
15    10716608
pg     6524057
18     5359151
12     3387443
Name: RATING, dtype: int64

In [16]:
# Drop any nan users or editorial ids
data = data[(~data['ID_USER'].isnull()) & (~data['ID_EDITORIAL'].isnull())]

In [18]:
data.to_csv('s3://bt-data-science-playground/bt-tv-recommendation-system/model_objects/historicalviews'+_v+'.csv', index=False)